In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import seaborn as sns
import glob
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
RootPath = '/content/drive/MyDrive/uOttawa/AI_for_CyberSecurity/InsiderThreatDetection'
AnswersPATH = os.path.join(RootPath,'answers')
DataSetPath = os.path.join(RootPath,'sampled dataset')

In [ ]:
DATASET_VER = 5.2
insiders_df = pd.read_csv(os.path.join(AnswersPATH,"insiders.csv"))
insiders_df = insiders_df[(insiders_df['dataset'] == DATASET_VER)]
insiders_df['start'] = pd.to_datetime(insiders_df['start'])
insiders_df = insiders_df.sort_values(by='start', ascending=True)
insiders_df.head()

,dataset,scenario,details,user,start,end
177,5.2,4,r5.2-4-OHS0036.csv,OHS0036,2010-04-16 12:32:57,07/09/2010 10:32:40
148,5.2,3,r5.2-3-MPF0690.csv,MPF0690,2010-06-17 10:00:38,06/18/2010 20:22:06
111,5.2,2,r5.2-2-VCF1602.csv,VCF1602,2010-06-23 15:32:14,08/11/2010 18:01:01
128,5.2,2,r5.2-2-CKP0630.csv,CKP0630,2010-06-28 08:13:30,08/13/2010 12:59:03
115,5.2,2,r5.2-2-ZIE0741.csv,ZIE0741,2010-06-29 13:16:09,08/27/2010 18:13:31


In [ ]:
# get user functional dictionary
all_files = glob.glob(os.path.join(DataSetPath,"LDAP/*.csv"))
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
LDAP_df = pd.concat(li, axis=0, ignore_index=True).drop_duplicates()

print(f'length of LDAP : {len(LDAP_df)}')
LDAP_df.head(10)

length of LDAP : 3581


,employee_name,user_id,email,role,projects,business_unit,functional_unit,department,team,supervisor
0,Maisie Maggy Kline,MMK1532,Maisie.Maggy.Kline@dtaa.com,Technician,NaN,2 - Executive,5 - SalesAndMarketing,3 - FieldService,3 - RegionalFieldService,Yvette Naomi Cortez
1,Nicole Tanya Burt,NTB0710,Nicole.Tanya.Burt@dtaa.com,SoftwareQualityEngineer,Project 2,2 - Executive,2 - ResearchAndEngineering,3 - SoftwareManagement,1 - SQA,Brennan Thaddeus Dominguez
2,Madeline Tatum Donaldson,MTD0971,Madeline.Tatum.Donaldson@dtaa.com,ElectricalEngineer,NaN,1 - Executive,2 - ResearchAndEngineering,3 - SoftwareManagement,6 - EmbeddedSoftware,Todd Rajah Peterson
3,Nicholas Hamish Browning,NHB1529,Nicholas.Hamish.Browning@dtaa.com,Technician,NaN,2 - Executive,5 - SalesAndMarketing,3 - FieldService,3 - RegionalFieldService,Yvette Naomi Cortez
4,Hoyt Berk Wiley,HBW0057,Hoyt.Berk.Wiley@dtaa.com,ProjectManager,NaN,1 - Executive,2 - ResearchAndEngineering,1 - ProjectManagement,NaN,Rina Adena Horn
5,Patience Lesley Mccoy,PLM1716,Patience.Lesley.Mccoy@dtaa.com,Salesman,NaN,2 - Executive,5 - SalesAndMarketing,2 - Sales,4 - RegionalSales,Ethan Branden Garrett
6,Ulric Kirk Guthrie,UKG0386,Ulric.Kirk.Guthrie@dtaa.com,ProductionLineWorker,NaN,2 - Executive,3 - Manufacturing,3 - Assembly,1 - AssemblyDept,Prescott Jasper Collins
7,Guinevere Lisandra Chang,GLC1055,Guinevere.Lisandra.Chang@dtaa.com,ProductionLineWorker,NaN,1 - Executive,3 - Manufacturing,3 - Assembly,4 - AssemblyDept,Evangeline Yoko Malone
8,Ulric Ciaran Dunlap,UCD0608,Ulric.Ciaran.Dunlap@dtaa.com,StockroomClerk,NaN,1 - Executive,3 - Manufacturing,3 - Assembly,1 - Stockroom,Sydney Genevieve Nichols
9,Coby Baxter Nguyen,CBN1983,Coby.Baxter.Nguyen@dtaa.com,LabManager,NaN,1 - Executive,2 - ResearchAndEngineering,2 - Research,6 - Lab,Dominique Karly Charles


##Features

In [ ]:
logon = pd.read_csv(os.path.join(RootPath,"New_features/user_daily_logon.csv"))
logon.head()

,day,user,count_Logon_working_hour_other_pc,count_Logon_working_hour_personal_pc,count_Logon_after_hours_other_pc,count_Logon_after_hours_personal_pc,count_Logon_after_hours_supervisor_pc,count_Logon_working_hour_supervisor_pc,session_time
0,2010-01-02,ELT1370,1.0,2.0,0.0,0.0,0.0,0.0,163.666667
1,2010-01-02,ESM1828,1.0,1.0,0.0,0.0,0.0,0.0,276.500000
2,2010-01-02,GVM1337,0.0,2.0,0.0,0.0,0.0,0.0,277.000000
3,2010-01-02,ILM1441,1.0,1.0,0.0,0.0,0.0,0.0,261.500000
4,2010-01-02,IRH1224,0.0,1.0,0.0,0.0,0.0,0.0,686.000000


In [ ]:
device = pd.read_csv(os.path.join(RootPath,"New_features/user_daily_device.csv"))
device.head()

,day,user,count_Connect_working_hour_personal_pc,count_Connect_after_hours_other_pc,count_Connect_working_hour_other_pc,count_Connect_after_hours_personal_pc,count_Connect_after_hours_supervisor_pc,count_Connect_working_hour_supervisor_pc,file_depth,connection_time
0,2010-01-02,ESM1828,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.00
1,2010-01-02,VRP0267,0.0,4.0,0.0,0.0,0.0,0.0,4.0,2.25
2,2010-01-03,KSS1005,5.0,0.0,0.0,0.0,0.0,0.0,5.0,7.80
3,2010-01-03,MAB1340,2.0,0.0,0.0,0.0,0.0,0.0,6.0,12.50
4,2010-01-03,VCF1602,5.0,0.0,0.0,0.0,0.0,0.0,2.0,10.20


In [ ]:
http = pd.read_csv(os.path.join(RootPath,"New_features/user_daily_http.csv"))
http.head()

,day,user,count_cloud_webvisit,count_hacktivist_webvisit,count_jobsearch_webvisit,count_social_webvisit,count_webvisit,url_length,url_depth,http_content_length
0,2010-01-02,ELT1370,0.0,0.0,0.0,1.0,9.0,87.500000,5.0,53.000000
1,2010-01-02,ESM1828,0.0,0.0,0.0,0.0,153.0,98.660131,5.0,49.117647
2,2010-01-02,GVM1337,2.0,0.0,0.0,2.0,91.0,95.705263,5.0,50.042105
3,2010-01-02,ILM1441,0.0,0.0,0.0,0.0,27.0,97.518519,5.0,47.740741
4,2010-01-02,IRH1224,0.0,0.0,0.0,9.0,86.0,84.547368,5.0,48.389474


In [ ]:
pd.read_csv(os.path.join(DataSetPath,"email.csv")).head()

,id,date,user,pc,to,cc,bcc,from,activity,size,attachments,content
0,{Y4F7-N0DM86BL-9295AJOV},01/02/2010 07:47:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,5353488,13.doc(1835701);army.doc(1931135);covered.jpg(...,remind sought suffered onto conference establi...
1,{O6W5-S9OE61YZ-9954AAJV},01/02/2010 07:48:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Receive,5353488,13.doc(1835701);army.doc(1931135);covered.jpg(...,remind sought suffered onto conference establi...
2,{A2G6-T7FE08CK-7601IYDT},01/02/2010 07:50:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Send,22721,NaN,warned repair devastated twin passing french h...
3,{V9E1-E4KG05KT-8603JHAW},01/02/2010 07:50:40,GVM1337,PC-6338,Lara.Cathleen.Rich@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,86054,NaN,repulsed francis everything three rationalised...
4,{W8E0-S2GA51NB-8855DBRL},01/02/2010 07:51:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Receive,22721,NaN,warned repair devastated twin passing french h...


In [ ]:
email = pd.read_csv(os.path.join(RootPath,"New_features/user_daily_email.csv"))
email.head()

,day,user,count_Receive_working_hour_personal_pc_0_0,count_Send_working_hour_personal_pc_0_0,count_Receive_working_hour_personal_pc_1_0,count_Send_working_hour_personal_pc_1_0,count_Send_working_hour_personal_pc_1_1,count_Receive_after_hours_personal_pc_0_0,count_Receive_after_hours_personal_pc_1_0,count_Send_after_hours_personal_pc_0_0,...,count_Send_working_hour_other_pc_1_0,count_Send_after_hours_other_pc_1_0,no_destinations,no_attachments,no_attachments_type,attachments_size,no_external_dest,no_bcc_dest,no_words,size
0,2010-01-02,ELT1370,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.500000,61.000000,45721.000000
1,2010-01-02,ESM1828,5.0,5.0,12.0,7.0,4.0,0.0,0.0,0.0,...,0.0,0.0,1.666667,0.363636,0.363636,307439.090909,0.939394,0.242424,47.727273,336874.696970
2,2010-01-02,GVM1337,5.0,4.0,5.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.526316,0.842105,0.631579,706252.105263,0.526316,0.263158,49.368421,739370.315789
3,2010-01-02,HJB1444,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.500000,0.500000,0.500000,135437.500000,0.500000,0.000000,42.500000,161672.500000
4,2010-01-02,ILM1441,1.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.333333,0.666667,0.666667,306603.333333,1.000000,0.500000,37.666667,330746.666667


In [ ]:
file = pd.read_csv(os.path.join(RootPath,"New_features/user_daily_file.csv"))
file.head()

,day,user,count_File Open_doc_False_after_hours_other_pc,count_File Open_doc_True_after_hours_other_pc,count_File Write_pdf_False_after_hours_other_pc,count_File Open_doc_True_working_hour_personal_pc,count_File Write_doc_False_working_hour_personal_pc,count_File Open_txt_True_working_hour_personal_pc,count_File Open_pdf_True_working_hour_other_pc,count_File Write_doc_False_working_hour_other_pc,...,count_File Open_exe_True_after_hours_supervisor_pc,count_File Open_txt_False_after_hours_personal_pc,count_File Open_zip_True_after_hours_personal_pc,count_File Write_zip_False_after_hours_personal_pc,count_File Open_jpg_False_after_hours_personal_pc,count_File Write_exe_False_after_hours_personal_pc,count_File Open_exe_False_working_hour_other_pc,count_File Open_exe_True_working_hour_supervisor_pc,file_length,file_depth
0,2010-01-02,VRP0267,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.000000,2.666667
1,2010-01-03,MAB1340,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.250000,3.000000
2,2010-01-03,VCF1602,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.333333,3.000000
3,2010-01-03,VRP0267,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.500000,2.833333
4,2010-01-04,AES1373,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.100000,2.700000


#**General Features**

In [ ]:
from datetime import timedelta

def is_weekend(x):
    if (x == 5) or (x==6):
        return "weekend"
    else:
        return "weekday"

def is_working_hour(x):
    if(x >= 7 and x <= 18):
        return 'working_hour'
    else:
        return "after_hours"
    
def get_user_pc(x, unique_pcs):
    '''
    '''
    user = unique_pcs[unique_pcs['pc'] == x['pc']].user.values
    if(len(user)):
        return user[0]

def get_supervisor(user_id, ldap_file=LDAP_df):
    supervisor_name = ldap_file[ldap_file['user_id'] == user_id].supervisor.values[0]
    supervisor_id = ldap_file[ldap_file['employee_name'] == supervisor_name].user_id
    if(len(supervisor_id)):
      return supervisor_id.values[0]
    else:
      return None

def is_personal_pc(x, user_pcs_df):
    '''
    '''
    user_pc = user_pcs_df[user_pcs_df['user'] == x.user].pc.values[0]
    
    if(user_pc == x.pc):
        return "personal_pc"
    
    supervisor = get_supervisor(x.user)
    if(supervisor is not None):
      supervisor_pc = user_pcs_df[user_pcs_df['user'] == supervisor].pc.values[0]
      if(supervisor_pc == x.pc):
          return "supervisor_pc"
      else:
          return "other_pc"
    return "other_pc"
    
def map_user_pcs(df, time_period=15):
    '''
    '''
    user_pc_df = pd.DataFrame(columns=['pc', 'user'])
    start_date = df.date.min()
    end_date =  start_date + timedelta(days=time_period)
    
    time_period_logons = df[(df['date'] > start_date) & (df['date'] < end_date)]

    user_pc_df['pc'] = time_period_logons['pc'].unique()
    user_pc_counts = time_period_logons.groupby(['user', 'pc']).size()
    user_pc_counts = user_pc_counts.to_frame(name = 'count').reset_index()
    idxs = user_pc_counts.groupby(['user'])['count'].transform(max) == user_pc_counts['count']
    unique_pcs = user_pc_counts[idxs]
    user_pc_df['user'] = user_pc_df.apply(get_user_pc, args=[unique_pcs], axis=1)
    
    return user_pc_df.dropna()

In [ ]:
all_logon_df = pd.read_csv(os.path.join(RootPath,"logon.csv"))
all_logon_df['date'] = pd.to_datetime(all_logon_df['date'])
all_logon_df.head()

,id,date,user,pc,activity
0,{Q4D5-W4HH44UC-5188LWZK},2010-01-02 02:24:51,JBI1134,PC-0168,Logon
1,{G7V0-S4TP95SA-9203AOGR},2010-01-02 02:38:28,JBI1134,PC-0168,Logoff
2,{B4U7-K4DB84LM-2657VBFY},2010-01-02 04:55:52,JBI1134,PC-2320,Logon
3,{K2E1-W7VG04OA-3686THSV},2010-01-02 05:02:28,JBI1134,PC-2320,Logoff
4,{Z7B2-Z8AS75YE-4485NAFE},2010-01-02 06:35:00,HMI1448,PC-9352,Logon


In [ ]:
user_pcs_df = map_user_pcs(all_logon_df)
user_pcs_df

,pc,user
0,PC-0168,KKP0064
1,PC-2320,ADR1156
2,PC-9352,HMI1448
3,PC-5439,JEH0587
4,PC-2683,KMC1934
...,...,...
1997,PC-3634,FWB1999
1998,PC-3877,IRN1736
1999,PC-4325,MSK0117
2000,PC-0845,PBS0120


#**Email**

In [ ]:
email_df = pd.read_csv(os.path.join(DataSetPath,'email.csv'))
email_df['date'] = pd.to_datetime(email_df['date'])
email_df.head(5)

,id,date,user,pc,to,cc,bcc,from,activity,size,attachments,content
0,{Y4F7-N0DM86BL-9295AJOV},2010-01-02 07:47:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,5353488,13.doc(1835701);army.doc(1931135);covered.jpg(...,remind sought suffered onto conference establi...
1,{O6W5-S9OE61YZ-9954AAJV},2010-01-02 07:48:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Receive,5353488,13.doc(1835701);army.doc(1931135);covered.jpg(...,remind sought suffered onto conference establi...
2,{A2G6-T7FE08CK-7601IYDT},2010-01-02 07:50:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Send,22721,NaN,warned repair devastated twin passing french h...
3,{V9E1-E4KG05KT-8603JHAW},2010-01-02 07:50:40,GVM1337,PC-6338,Lara.Cathleen.Rich@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,86054,NaN,repulsed francis everything three rationalised...
4,{W8E0-S2GA51NB-8855DBRL},2010-01-02 07:51:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Receive,22721,NaN,warned repair devastated twin passing french h...


In [ ]:
# statistical features
def get_no_attachments_type(attachment):
  if (attachment != attachment): return 0

  types = [re.findall('\.[a-zA-Z0-9]*\(',i)[0][1:-1]for i in attachment.split(';')]
  unique_types = np.unique(np.array(types))
  return len(unique_types)

def get_attachments_size(attachment):
  if (attachment != attachment): return 0

  sizes = [int(re.findall('\([0-9]+\)',i)[0][1:-1])for i in attachment.split(';')]
  #sizes = np.array(sizes)/1024/1024
  sum_sizes = np.sum(np.array(sizes))
  return sum_sizes

def get_no_external_dest(to):
  dests_count = len(to.split(';'))
  internal_count = len(re.findall('@dtaa.com',to))
  return dests_count - internal_count

def is_external_mail(to):
    if to != to : return False

    dests_count = len(to.split(';'))
    internal_count = len(re.findall('@dtaa.com',to))
    return (dests_count - internal_count)>0

def process_email(e_df):
  df = e_df.copy()
  # statistical features
  df['no_destinations'] = df['to'].apply(lambda x : len(x.split(';')))
  df['no_attachments'] = df['attachments'].apply(lambda x : 0 if x != x else len(x.split(';')))
  df['no_attachments_type'] = df['attachments'].apply(get_no_attachments_type)
  df['attachments_size'] = df['attachments'].apply(get_attachments_size)
  df['no_external_dest'] = df['to'].apply(get_no_external_dest)
  df['no_bcc_dest'] = df['bcc'].apply(lambda x : 0 if x != x else len(x.split(';')))
  df['no_words'] = df['content'].apply(lambda x : 0 if x != x else len(x.split(' ')))
  # df['size'] = df['size'].apply(lambda x : x/1024/1024)

  # count features
  df['is_external'] = df['to'].apply(lambda x :1 if is_external_mail(x) else 0)
  df['is_external_bcc'] = df['bcc'].apply(lambda x :1 if is_external_mail(x) else 0)

  return df

In [ ]:
tmp_email_df = process_email(email_df)
tmp_email_df['weekday'] = tmp_email_df.date.dt.weekday
tmp_email_df['hour'] = tmp_email_df['date'].dt.hour
tmp_email_df['day'] = pd.to_datetime(tmp_email_df['date']).dt.normalize()

tmp_email_df['is_weekend'] = tmp_email_df['weekday'].apply(is_weekend)
tmp_email_df['is_working_hour'] = tmp_email_df['hour'].apply(is_working_hour)
tmp_email_df['is_personal_pc'] = tmp_email_df.apply(is_personal_pc, args=[user_pcs_df], axis=1)

tmp_email_df.head(10)

,id,date,user,pc,to,cc,bcc,from,activity,size,...,no_bcc_dest,no_words,is_external,is_external_bcc,weekday,hour,day,is_weekend,is_working_hour,is_personal_pc
0,{Y4F7-N0DM86BL-9295AJOV},2010-01-02 07:47:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,5353488,...,1,70,0,0,5,7,2010-01-02,weekend,working_hour,personal_pc
1,{O6W5-S9OE61YZ-9954AAJV},2010-01-02 07:48:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Receive,5353488,...,0,70,0,0,5,7,2010-01-02,weekend,working_hour,personal_pc
2,{A2G6-T7FE08CK-7601IYDT},2010-01-02 07:50:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Send,22721,...,0,44,1,0,5,7,2010-01-02,weekend,working_hour,personal_pc
3,{V9E1-E4KG05KT-8603JHAW},2010-01-02 07:50:40,GVM1337,PC-6338,Lara.Cathleen.Rich@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,86054,...,1,69,0,0,5,7,2010-01-02,weekend,working_hour,personal_pc
4,{W8E0-S2GA51NB-8855DBRL},2010-01-02 07:51:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Receive,22721,...,0,44,1,0,5,7,2010-01-02,weekend,working_hour,personal_pc
5,{V9Y4-Y6YA45RC-6762BMXK},2010-01-02 07:51:40,GVM1337,PC-6338,Lara.Cathleen.Rich@dtaa.com,NaN,NaN,Gregory.Victor.Mccarty@dtaa.com,Receive,86054,...,0,69,0,0,5,7,2010-01-02,weekend,working_hour,personal_pc
6,{F3S0-B5AP99TB-4530VMJO},2010-01-02 08:03:57,IRH1224,PC-4126,Ethan.B.Garrett@yahoo.com;Stewart.Kennan@cox.net,LSM14@charter.net,Ivana.R.Holland@optonline.net,Ivana.R.Holland@optonline.net,Send,26210,...,1,50,1,1,5,8,2010-01-02,weekend,working_hour,personal_pc
7,{L3U3-N0SK79WC-6849WIHK},2010-01-02 08:04:04,IRH1224,PC-4126,Ethan.B.Garrett@yahoo.com,Ivana.R.Holland@optonline.net,NaN,Ivana.R.Holland@optonline.net,Send,43652,...,0,59,1,0,5,8,2010-01-02,weekend,working_hour,personal_pc
8,{U1X2-Z4AU52HO-7851KWYP},2010-01-02 08:04:06,IRH1224,PC-4126,Ethan.B.Garrett@yahoo.com,NaN,Ivana.R.Holland@optonline.net,Ivana.R.Holland@optonline.net,Send,29059,...,1,39,1,1,5,8,2010-01-02,weekend,working_hour,personal_pc
9,{T1M8-A5KW20CX-6235QRKK},2010-01-02 08:04:57,IRH1224,PC-4126,Ethan.B.Garrett@yahoo.com;Stewart.Kennan@cox.net,LSM14@charter.net,NaN,Ivana.R.Holland@optonline.net,Receive,26210,...,0,50,1,0,5,8,2010-01-02,weekend,working_hour,personal_pc


**Count Features**

In [ ]:
groupby_cols = ['day', 'user', 'activity', 'is_working_hour', 'is_personal_pc' ,'is_external' , 'is_external_bcc']
user_daily_email_counts = pd.DataFrame({"count" : tmp_email_df.groupby(groupby_cols).size()}).reset_index()
user_daily_email_counts = user_daily_email_counts.set_index(groupby_cols).unstack(['activity', 'is_working_hour', 'is_personal_pc' , 'is_external' , 'is_external_bcc'])
user_daily_email_counts.columns = ['{}_{}_{}_{}_{}_{}'.format(a,b,c,d,e,f) for a,b,c,d,e,f in user_daily_email_counts.columns]

user_daily_email_counts.fillna(0, inplace=True)
user_daily_email_counts = user_daily_email_counts.reset_index()
user_daily_email_counts

,day,user,count_Receive_working_hour_personal_pc_0_0,count_Send_working_hour_personal_pc_0_0,count_Receive_working_hour_personal_pc_1_0,count_Send_working_hour_personal_pc_1_0,count_Send_working_hour_personal_pc_1_1,count_Receive_after_hours_personal_pc_0_0,count_Receive_after_hours_personal_pc_1_0,count_Send_after_hours_personal_pc_0_0,count_Send_after_hours_personal_pc_1_0,count_Send_after_hours_personal_pc_1_1,count_Send_working_hour_other_pc_1_0,count_Send_after_hours_other_pc_1_0
0,2010-01-02,ELT1370,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-02,ESM1828,5.0,5.0,12.0,7.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-01-02,GVM1337,5.0,4.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-01-02,HJB1444,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-02,ILM1441,1.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62464,2011-06-01,WBK1077,18.0,5.0,21.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62465,2011-06-01,WCW1013,19.0,8.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62466,2011-06-01,WOS1834,11.0,5.0,12.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62467,2011-06-01,YBD1929,7.0,8.0,9.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Statistical Features**

In [ ]:
user_daily_email_stats = tmp_email_df.groupby(["day", "user"])['no_destinations', 'no_attachments','no_attachments_type',
                         'attachments_size','no_external_dest','no_bcc_dest','no_words','size'].mean().reset_index()

user_daily_email_stats.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,day,user,no_destinations,no_attachments,no_attachments_type,attachments_size,no_external_dest,no_bcc_dest,no_words,size
0,2010-01-02,ELT1370,1.000000,0.000000,0.000000,0.000000,0.000000,0.500000,61.000000,45721.000000
1,2010-01-02,ESM1828,1.666667,0.363636,0.363636,307439.090909,0.939394,0.242424,47.727273,336874.696970
2,2010-01-02,GVM1337,1.526316,0.842105,0.631579,706252.105263,0.526316,0.263158,49.368421,739370.315789
3,2010-01-02,HJB1444,1.500000,0.500000,0.500000,135437.500000,0.500000,0.000000,42.500000,161672.500000
4,2010-01-02,ILM1441,1.333333,0.666667,0.666667,306603.333333,1.000000,0.500000,37.666667,330746.666667
5,2010-01-02,IRH1224,1.444444,0.333333,0.333333,247926.888889,1.000000,0.333333,50.000000,280748.222222
6,2010-01-02,JMM0613,3.000000,0.000000,0.000000,0.000000,1.000000,0.500000,54.500000,25440.000000
7,2010-01-02,KRC0098,1.833333,0.555556,0.500000,501808.555556,0.277778,0.250000,52.555556,533078.666667
8,2010-01-02,VRP0267,1.833333,0.333333,0.333333,133522.916667,0.458333,0.458333,48.916667,167208.583333
9,2010-01-03,DLW1705,1.850000,0.650000,0.600000,491340.100000,0.500000,0.000000,44.650000,519133.350000


**Merge**

In [ ]:
user_daily_email = user_daily_email_counts.merge(user_daily_email_stats, on=['day', 'user'])
user_daily_email.head(10)

,day,user,count_Receive_working_hour_personal_pc_0_0,count_Send_working_hour_personal_pc_0_0,count_Receive_working_hour_personal_pc_1_0,count_Send_working_hour_personal_pc_1_0,count_Send_working_hour_personal_pc_1_1,count_Receive_after_hours_personal_pc_0_0,count_Receive_after_hours_personal_pc_1_0,count_Send_after_hours_personal_pc_0_0,...,count_Send_working_hour_other_pc_1_0,count_Send_after_hours_other_pc_1_0,no_destinations,no_attachments,no_attachments_type,attachments_size,no_external_dest,no_bcc_dest,no_words,size
0,2010-01-02,ELT1370,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.500000,61.000000,45721.000000
1,2010-01-02,ESM1828,5.0,5.0,12.0,7.0,4.0,0.0,0.0,0.0,...,0.0,0.0,1.666667,0.363636,0.363636,307439.090909,0.939394,0.242424,47.727273,336874.696970
2,2010-01-02,GVM1337,5.0,4.0,5.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.526316,0.842105,0.631579,706252.105263,0.526316,0.263158,49.368421,739370.315789
3,2010-01-02,HJB1444,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.500000,0.500000,0.500000,135437.500000,0.500000,0.000000,42.500000,161672.500000
4,2010-01-02,ILM1441,1.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.333333,0.666667,0.666667,306603.333333,1.000000,0.500000,37.666667,330746.666667
5,2010-01-02,IRH1224,2.0,2.0,7.0,3.0,4.0,0.0,0.0,0.0,...,0.0,0.0,1.444444,0.333333,0.333333,247926.888889,1.000000,0.333333,50.000000,280748.222222
6,2010-01-02,JMM0613,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,3.000000,0.000000,0.000000,0.000000,1.000000,0.500000,54.500000,25440.000000
7,2010-01-02,KRC0098,16.0,16.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.833333,0.555556,0.500000,501808.555556,0.277778,0.250000,52.555556,533078.666667
8,2010-01-02,VRP0267,10.0,8.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.833333,0.333333,0.333333,133522.916667,0.458333,0.458333,48.916667,167208.583333
9,2010-01-03,DLW1705,0.0,14.0,4.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.850000,0.650000,0.600000,491340.100000,0.500000,0.000000,44.650000,519133.350000


In [ ]:
user_daily_email.to_csv(os.path.join(RootPath,'user_daily_email.csv'),index = False)

'/content/drive/MyDrive/AI_for_CyberSecurity/InsiderThreatDetection'

###**trails**

In [ ]:
tmp = email_df.copy()

In [ ]:
tmp['content'].values[0]

'remind sought suffered onto conference established next parliamentary denis underframes he contract than expressed great said pepler curve sign move county spring privates what diefenbaker level see eastern march william km edmonton shielded such sat sun responsibility mainline injuries bringing missing journey lengthy pointed australian 13 rescuers covered ground army the demolished ad forestry brought dining find having honour ad moderate biographer green million struck won an death judge junction'

In [ ]:

tmp['attachments'].values[0] != tmp['attachments'].values[0]

False

In [ ]:
attachment = tmp['attachments'].iloc[0]
attachment

'13.doc(1835701);army.doc(1931135);covered.jpg(857454);parliamentary.pdf(50720);next.doc(646408)'

In [ ]:
import re
# number of unique attachments
a  = attachment.split(';')
b = [re.findall('\.[a-zA-Z0-9]*\(',i)[0][1:-1]for i in a]
print(b)
print(np.unique(np.array(b)))
len(np.unique(np.array(b)))
# len(np.unique(np.array([re.findall('\.[a-zA-Z0-9]*\(',i)[0][1:-1]for i in a])))

['doc', 'doc', 'jpg', 'pdf', 'doc']
['doc' 'jpg' 'pdf']


3

In [ ]:
# attachment_size
a  = attachment.split(';')
b = [int(re.findall('\([0-9]+\)',i)[0][1:-1])for i in a]
np.sum(np.array(b))

np.sum(np.array([int(re.findall('\([0-9]+\)',i)[0][1:-1])for i in attachment.split(';')]))

5321418

In [ ]:
# statistical features
tmp['no_destinations'] = tmp['to'].apply(lambda x : len(x.split(';')))
tmp['no_attachments'] = tmp['attachments'].apply(lambda x : 0 if x != x else len(x.split(';')))
tmp['no_attachments_type'] = tmp['attachments'].apply(get_no_attachments_type)
tmp['attachments_size'] = tmp['attachments'].apply(get_attachments_size)
tmp['no_external_dest'] = tmp['to'].apply(get_no_external_dest)
tmp['no_bcc_dest'] = tmp['bcc'].apply(lambda x : 0 if x != x else len(x.split(';')))
tmp['no_words'] = tmp['content'].apply(lambda x : 0 if x != x else len(x.split(' ')))
tmp['size'] = tmp['size'].apply(lambda x : x/1024/1024)

# count features
tmp['external'] = tmp['to'].apply(lambda x :1 if is_external_mail(x) else 0)
tmp['external_bcc'] = tmp['bcc'].apply(lambda x :1 if is_external_mail(x) else 0)

In [ ]:
tmp.head()

,id,date,user,pc,to,cc,bcc,from,activity,size,...,content,no_destinations,no_attachments,no_attachments_type,attachments_size,no_external_dest,no_bcc_dest,no_words,external,external_bcc
0,{Y4F7-N0DM86BL-9295AJOV},2010-01-02 07:47:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,5.105484,...,remind sought suffered onto conference establi...,2,5,3,5.0749,0,1,70,0,0
1,{O6W5-S9OE61YZ-9954AAJV},2010-01-02 07:48:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Receive,5.105484,...,remind sought suffered onto conference establi...,2,5,3,5.0749,0,0,70,0,0
2,{A2G6-T7FE08CK-7601IYDT},2010-01-02 07:50:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Send,0.021668,...,warned repair devastated twin passing french h...,1,0,0,0.0000,1,0,44,1,0
3,{V9E1-E4KG05KT-8603JHAW},2010-01-02 07:50:40,GVM1337,PC-6338,Lara.Cathleen.Rich@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,0.082067,...,repulsed francis everything three rationalised...,1,0,0,0.0000,0,1,69,0,0
4,{W8E0-S2GA51NB-8855DBRL},2010-01-02 07:51:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Receive,0.021668,...,warned repair devastated twin passing french h...,1,0,0,0.0000,1,0,44,1,0


#**Device**

I should check if the connection time exceeds 720 (half day)--> replce it by end_of_day-connection_start_time if less than 720 else keep 720

In [ ]:
device_df = pd.read_csv(os.path.join(DataSetPath,'device.csv'))
device_df['date'] = pd.to_datetime(device_df['date'])
device_df.head(5)

,id,date,user,pc,file_tree,activity
0,{I3E7-Z8KN26QC-0847RGRG},2010-01-02 08:42:18,ESM1828,PC-7721,R:\;R:\ESM1828;R:\584Xlk8,Connect
1,{Q0Z2-I6OI75TP-7558HMVN},2010-01-02 08:44:11,ESM1828,PC-7721,NaN,Disconnect
2,{E8D8-X5NB52VQ-5232FIDW},2010-01-02 19:13:36,VRP0267,PC-8461,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect
3,{B1E2-Z3MS26GZ-5290QMFQ},2010-01-02 19:15:49,VRP0267,PC-8461,NaN,Disconnect
4,{P7V4-W2QB19PV-4652XYPX},2010-01-02 22:02:22,VRP0267,PC-4916,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect


In [ ]:
# statistical features
def add_connection_time(dev_df):
  grouped_df = dev_df.groupby(['user', 'pc'])
  df_list = []

  for key,_ in grouped_df:
    group = grouped_df.get_group(key)
    group = group.sort_values(['date'] , ascending = [False])

    difference = group["date"].diff().apply(lambda x: x/np.timedelta64(1, 'm')*-1).fillna(0).astype('int64')
    group['connection_time'] = pd.Series(difference)
    group = group[group['activity']=='Connect']

    df_list.append(group)

  device_df_time = pd.concat(df_list, ignore_index = True, axis = 0)

  return device_df_time

def get_file_tree_length(path):
  if path != path : return 0
  return len(path.split(';'))


def process_device(dev_df):
  df = dev_df.copy()
  dev_df_time = add_connection_time(df)
  dev_df_time['file_depth'] =  dev_df_time['file_tree'].apply(get_file_tree_length)

  return dev_df_time

In [ ]:
tmp_device_df = process_device(device_df)
tmp_device_df['weekday'] = tmp_device_df.date.dt.weekday
tmp_device_df['hour'] = tmp_device_df['date'].dt.hour
tmp_device_df['day'] = pd.to_datetime(tmp_device_df['date']).dt.normalize()

tmp_device_df['is_weekend'] = tmp_device_df['weekday'].apply(is_weekend)
tmp_device_df['is_working_hour'] = tmp_device_df['hour'].apply(is_working_hour)
tmp_device_df['is_personal_pc'] = tmp_device_df.apply(is_personal_pc, args=[user_pcs_df], axis=1)

tmp_device_df.head(10)

,id,date,user,pc,file_tree,activity,connection_time,file_depth,weekday,hour,day,is_weekend,is_working_hour,is_personal_pc
0,{P1C4-W8RA33YS-4173HKFX},2010-12-02 20:25:38,ACA1126,PC-1124,R:\;R:\ACA1126;R:\93P82N6,Connect,6,3,3,20,2010-12-02,weekday,after_hours,supervisor_pc
1,{A9G8-R2LA36ZL-2784ZTEV},2010-12-02 15:35:54,ACA1126,PC-1737,R:\;R:\ACA1126;R:\93P82N6,Connect,12,3,3,15,2010-12-02,weekday,working_hour,personal_pc
2,{U3N5-B3GG22IH-8708IAOY},2011-05-31 14:45:46,AES1373,PC-6944,R:\;R:\AES1373,Connect,54,2,1,14,2011-05-31,weekday,working_hour,personal_pc
3,{O4O9-F9FA85KI-0399EPGH},2011-05-31 10:42:06,AES1373,PC-6944,R:\;R:\AES1373,Connect,15,2,1,10,2011-05-31,weekday,working_hour,personal_pc
4,{N1J2-U0KJ64RK-2745DCQD},2011-05-31 10:09:02,AES1373,PC-6944,R:\;R:\AES1373,Connect,13,2,1,10,2011-05-31,weekday,working_hour,personal_pc
5,{I1H2-J4IL07BS-3153CHYV},2011-05-27 14:12:13,AES1373,PC-6944,R:\;R:\AES1373,Connect,0,2,4,14,2011-05-27,weekday,working_hour,personal_pc
6,{W4V3-E6BF87PX-8374RPGO},2011-05-26 14:25:19,AES1373,PC-6944,R:\;R:\AES1373,Connect,6,2,3,14,2011-05-26,weekday,working_hour,personal_pc
7,{M8R4-L5XY71RX-4803FBAB},2011-05-25 13:35:56,AES1373,PC-6944,R:\;R:\AES1373,Connect,6,2,2,13,2011-05-25,weekday,working_hour,personal_pc
8,{Z2V2-Y7QN19IU-3595FPOT},2011-05-25 11:18:56,AES1373,PC-6944,R:\;R:\AES1373,Connect,57,2,2,11,2011-05-25,weekday,working_hour,personal_pc
9,{R1Y6-P3LE30MF-3387IHJX},2011-05-25 09:21:28,AES1373,PC-6944,R:\;R:\AES1373,Connect,45,2,2,9,2011-05-25,weekday,working_hour,personal_pc


In [ ]:
groupby_cols = ['day', 'user', 'activity', 'is_working_hour', 'is_personal_pc' ]
user_daily_device_counts = pd.DataFrame({"count" : tmp_device_df.groupby(groupby_cols).size()}).reset_index()
user_daily_device_counts = user_daily_device_counts.set_index(groupby_cols).unstack(['activity', 'is_working_hour', 'is_personal_pc'])
user_daily_device_counts.columns = ['{}_{}_{}_{}'.format(a,b,c,d) for a,b,c,d in user_daily_device_counts.columns]

user_daily_device_counts.fillna(0, inplace=True)
user_daily_device_counts = user_daily_device_counts.reset_index()
user_daily_device_counts

,day,user,count_Connect_working_hour_personal_pc,count_Connect_after_hours_other_pc,count_Connect_working_hour_other_pc,count_Connect_after_hours_personal_pc,count_Connect_after_hours_supervisor_pc,count_Connect_working_hour_supervisor_pc
0,2010-01-02,ESM1828,1.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-02,VRP0267,0.0,4.0,0.0,0.0,0.0,0.0
2,2010-01-03,KSS1005,5.0,0.0,0.0,0.0,0.0,0.0
3,2010-01-03,MAB1340,2.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-03,VCF1602,5.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
12484,2011-06-01,QEL1515,1.0,0.0,0.0,0.0,0.0,0.0
12485,2011-06-01,RCG0584,1.0,0.0,0.0,0.0,0.0,0.0
12486,2011-06-01,RHG0882,2.0,0.0,0.0,0.0,0.0,0.0
12487,2011-06-01,SMK1323,9.0,0.0,0.0,0.0,0.0,0.0


**Statistical Features**

In [ ]:
user_daily_device_stats = tmp_device_df.groupby(["day", "user"])['file_depth', 'connection_time'].mean().reset_index()
user_daily_device_stats.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,day,user,file_depth,connection_time
0,2010-01-02,ESM1828,3.0,1.000
1,2010-01-02,VRP0267,4.0,2.250
2,2010-01-03,KSS1005,5.0,7.800
3,2010-01-03,MAB1340,6.0,12.500
4,2010-01-03,VCF1602,2.0,10.200
5,2010-01-03,VRP0267,4.0,1.000
6,2010-01-04,AES1373,2.0,8.000
7,2010-01-04,ASR0150,3.0,14.125
8,2010-01-04,BYO1846,5.0,23.000
9,2010-01-04,CHP1711,3.0,22.600


**Merge**

In [ ]:
user_daily_device = user_daily_device_counts.merge(user_daily_device_stats, on=['day', 'user'])
user_daily_device.head(10)

,day,user,count_Connect_working_hour_personal_pc,count_Connect_after_hours_other_pc,count_Connect_working_hour_other_pc,count_Connect_after_hours_personal_pc,count_Connect_after_hours_supervisor_pc,count_Connect_working_hour_supervisor_pc,file_depth,connection_time
0,2010-01-02,ESM1828,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.000
1,2010-01-02,VRP0267,0.0,4.0,0.0,0.0,0.0,0.0,4.0,2.250
2,2010-01-03,KSS1005,5.0,0.0,0.0,0.0,0.0,0.0,5.0,7.800
3,2010-01-03,MAB1340,2.0,0.0,0.0,0.0,0.0,0.0,6.0,12.500
4,2010-01-03,VCF1602,5.0,0.0,0.0,0.0,0.0,0.0,2.0,10.200
5,2010-01-03,VRP0267,0.0,0.0,1.0,0.0,0.0,0.0,4.0,1.000
6,2010-01-04,AES1373,3.0,0.0,0.0,0.0,0.0,0.0,2.0,8.000
7,2010-01-04,ASR0150,8.0,0.0,0.0,0.0,0.0,0.0,3.0,14.125
8,2010-01-04,BYO1846,2.0,0.0,0.0,0.0,0.0,0.0,5.0,23.000
9,2010-01-04,CHP1711,5.0,0.0,0.0,0.0,0.0,0.0,3.0,22.600


In [ ]:
len(user_daily_device[user_daily_device['connection_time'] > 720])

59

In [ ]:
user_daily_device.to_csv(os.path.join(RootPath,'user_daily_device.csv'),index = False)

###**Trails**

In [ ]:
device_tmp = device_df.copy()
device_tmp.head(10)

,id,date,user,pc,file_tree,activity
0,{I3E7-Z8KN26QC-0847RGRG},2010-01-02 08:42:18,ESM1828,PC-7721,R:\;R:\ESM1828;R:\584Xlk8,Connect
1,{Q0Z2-I6OI75TP-7558HMVN},2010-01-02 08:44:11,ESM1828,PC-7721,NaN,Disconnect
2,{E8D8-X5NB52VQ-5232FIDW},2010-01-02 19:13:36,VRP0267,PC-8461,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect
3,{B1E2-Z3MS26GZ-5290QMFQ},2010-01-02 19:15:49,VRP0267,PC-8461,NaN,Disconnect
4,{P7V4-W2QB19PV-4652XYPX},2010-01-02 22:02:22,VRP0267,PC-4916,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect
5,{U3W6-Y4AN87OS-1151TVPC},2010-01-02 22:02:24,VRP0267,PC-4916,NaN,Disconnect
6,{Q4J2-G5OI74UN-8754XPXQ},2010-01-02 22:30:16,VRP0267,PC-0530,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect
7,{B0A3-P2HF80HY-7891KDTB},2010-01-02 22:34:07,VRP0267,PC-0530,NaN,Disconnect
8,{A2T3-G5FZ13ES-8545NEIL},2010-01-02 23:05:47,VRP0267,PC-1985,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect
9,{Y8C8-W8AW08WZ-7256JHIP},2010-01-02 23:10:27,VRP0267,PC-1985,NaN,Disconnect


In [ ]:
# features : connect duration, file tree length
device_tmp.sort_values(['user', 'pc','date'] , ascending = [False, False,True])

,id,date,user,pc,file_tree,activity
81538,{Z8O8-S8VG18DU-4498RVEC},2011-01-11 02:26:12,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect
81539,{H2P1-V9CT93ES-5492PNSX},2011-01-11 02:32:38,ZKP0542,PC-2083,NaN,Disconnect
81540,{T1X3-R8VM32JD-4082IPBP},2011-01-11 03:06:22,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect
81541,{V2U9-P6IN99DF-9466SHDO},2011-01-11 03:10:35,ZKP0542,PC-2083,NaN,Disconnect
81542,{O9M2-A0LZ88NQ-8497GZNB},2011-01-11 03:35:11,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect
...,...,...,...,...,...,...
94679,{C1M3-S2CZ35RP-2793SCUY},2011-05-31 15:40:13,AES1373,PC-6944,NaN,Disconnect
76602,{A9G8-R2LA36ZL-2784ZTEV},2010-12-02 15:35:54,ACA1126,PC-1737,R:\;R:\ACA1126;R:\93P82N6,Connect
76609,{M0N5-G4LS65PJ-1590BJXO},2010-12-02 15:48:39,ACA1126,PC-1737,NaN,Disconnect
76644,{P1C4-W8RA33YS-4173HKFX},2010-12-02 20:25:38,ACA1126,PC-1124,R:\;R:\ACA1126;R:\93P82N6,Connect


In [ ]:
device_tmp.loc[device_tmp.groupby(['user', 'pc']).date.idxmax()]

,id,date,user,pc,file_tree,activity
76645,{Q7A2-Z6XV01LX-3361JRPT},2010-12-02 20:32:02,ACA1126,PC-1124,NaN,Disconnect
76609,{M0N5-G4LS65PJ-1590BJXO},2010-12-02 15:48:39,ACA1126,PC-1737,NaN,Disconnect
94679,{C1M3-S2CZ35RP-2793SCUY},2011-05-31 15:40:13,AES1373,PC-6944,NaN,Disconnect
57801,{X0M6-O2CA69UT-6386EOTX},2010-08-24 05:48:51,ALT1465,PC-3407,R:\;R:\ALT1465,Connect
84909,{K1P5-C5LA22NV-5342TUJA},2011-02-09 03:22:51,ALW0764,PC-4156,NaN,Disconnect
...,...,...,...,...,...,...
89635,{N3F4-C2SK84LX-0912SQSI},2011-03-30 22:46:14,WWW0701,PC-6092,NaN,Disconnect
89755,{D4Z7-T4VG12WR-3447PGYH},2011-03-31 18:17:32,ZEH0685,PC-7195,NaN,Disconnect
89763,{H1D5-W9MS66JU-1110AXUW},2011-03-31 21:41:59,ZEH0685,PC-8814,NaN,Disconnect
59173,{W7B4-Q0QA62OT-0599UXNN},2010-08-27 18:13:31,ZIE0741,PC-9132,NaN,Disconnect


In [ ]:
# https://www.adamsmith.haus/python/answers/how-to-group-a-pandas-dataframe-by-multiple-columns-in-python

grouped_df = device_tmp.groupby(['user', 'pc'])
pd.DataFrame(grouped_df)

,0,1
0,"(ACA1126, PC-1124)",id ...
1,"(ACA1126, PC-1737)",id ...
2,"(AES1373, PC-6944)",id ...
3,"(ALT1465, PC-3407)",id ...
4,"(ALW0764, PC-4156)",id ...
...,...,...
1643,"(WWW0701, PC-6092)",id ...
1644,"(ZEH0685, PC-7195)",id ...
1645,"(ZEH0685, PC-8814)",id ...
1646,"(ZIE0741, PC-9132)",id ...


In [ ]:
a_group = grouped_df.get_group(('ALW0764', 'PC-4156'))
a_group = a_group.sort_values(['date'] , ascending = [False])

In [ ]:
# get difference in time measured by minuties (https://www.codeforests.com/2020/10/04/calculate-date-difference-between-rows/)
difference = a_group["date"].diff().apply(lambda x: x/np.timedelta64(1, 'm')*-1).fillna(0).astype('int64')
a_group['connection_time'] = pd.Series(difference)
a_group

,id,date,user,pc,file_tree,activity,connection_time
84909,{K1P5-C5LA22NV-5342TUJA},2011-02-09 03:22:51,ALW0764,PC-4156,NaN,Disconnect,0
84908,{R4Y6-O6LW61DX-9311KFGW},2011-02-09 02:24:00,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,58
84907,{H2A1-M1SV36CM-2637ZLPS},2011-02-09 02:06:42,ALW0764,PC-4156,NaN,Disconnect,17
84906,{B4P3-H9YL11NF-1340IZMO},2011-02-09 01:51:32,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,15
84578,{M8K7-T8BV96IS-8560WELG},2011-02-05 06:11:42,ALW0764,PC-4156,NaN,Disconnect,5499
84576,{U4G9-P8GK20LL-9697YCYF},2011-02-05 03:30:49,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,160
84432,{E4Q3-I4ST18NE-5212VTAT},2011-02-03 22:46:12,ALW0764,PC-4156,NaN,Disconnect,1724
84431,{X5S0-N7XG98BU-8011CGFG},2011-02-03 22:41:42,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,4
84427,{Z1I4-F3TJ54VH-7885SZRU},2011-02-03 22:24:56,ALW0764,PC-4156,NaN,Disconnect,16
84426,{P5Q3-P9LC27RO-4104FLLV},2011-02-03 22:15:45,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,9


In [ ]:
a_group[a_group['activity']=='Connect']

,id,date,user,pc,file_tree,activity,connection_time
84908,{R4Y6-O6LW61DX-9311KFGW},2011-02-09 02:24:00,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,58
84906,{B4P3-H9YL11NF-1340IZMO},2011-02-09 01:51:32,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,15
84576,{U4G9-P8GK20LL-9697YCYF},2011-02-05 03:30:49,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,160
84431,{X5S0-N7XG98BU-8011CGFG},2011-02-03 22:41:42,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,4
84426,{P5Q3-P9LC27RO-4104FLLV},2011-02-03 22:15:45,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,9
84275,{R1B9-O0TP44KT-2039ELXI},2011-02-03 07:12:36,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,16
83973,{V7U3-O6PS82RB-2789IYRP},2011-01-31 23:13:08,ALW0764,PC-4156,R:\;R:\ALW0764;R:\83p6vM4,Connect,11


In [ ]:
    tomorrow = dt + datetime.timedelta(days=1)
    return datetime.datetime.combine(tomorrow, datetime.time.min) - dt

In [ ]:
# def modify_connectiontime(row):
#   # features = [str(col)for col in row.index]
#   # row_df = pd.DataFrame([row.values], columns = features)
#   # print(row_df)

#   if row['connection_time'] >= 720 : 
#     connection_start = row['date']
#     end_day = connection_start + datetime.timedelta(days=1)
#     return datetime.datetime.combine(end_day, datetime.time.min) - connection_start

#   else:
#     return row['connection_time'] 

In [ ]:
# a_group['connection_time_modify'] = a_group.apply(modify_connectiontime)

In [ ]:
grouped_df = device_tmp.groupby(['user', 'pc'])
df_list = []

for key,_ in grouped_df:
  group = grouped_df.get_group(key)
  group = group.sort_values(['date'] , ascending = [False])

  difference = group["date"].diff().apply(lambda x: x/np.timedelta64(1, 'm')*-1).fillna(0).astype('int64')
  group['connection_time'] = pd.Series(difference)
  group = group[group['activity']=='Connect']

  df_list.append(group)

device_df_time = pd.concat(df_list, ignore_index = True, axis = 0)

In [ ]:
def get_connection_time(dev_df):

  grouped_df = dev_df.groupby(['user', 'pc'])
  df_list = []

  for key,_ in grouped_df:
    group = grouped_df.get_group(key)
    group = group.sort_values(['date'] , ascending = [False])

    difference = group["date"].diff().apply(lambda x: x/np.timedelta64(1, 'm')*-1).fillna(0).astype('int64')
    group['connection_time'] = pd.Series(difference)
    group = group[group['activity']=='Connect']

    df_list.append(group)

  device_df_time = pd.concat(df_list, ignore_index = True, axis = 0)

  return device_df_time

In [ ]:
device_df_time = get_connection_time(device_df)

In [ ]:
device_df_time

,id,date,user,pc,file_tree,activity,connection_time
0,{P1C4-W8RA33YS-4173HKFX},2010-12-02 20:25:38,ACA1126,PC-1124,R:\;R:\ACA1126;R:\93P82N6,Connect,6
1,{A9G8-R2LA36ZL-2784ZTEV},2010-12-02 15:35:54,ACA1126,PC-1737,R:\;R:\ACA1126;R:\93P82N6,Connect,12
2,{U3N5-B3GG22IH-8708IAOY},2011-05-31 14:45:46,AES1373,PC-6944,R:\;R:\AES1373,Connect,54
3,{O4O9-F9FA85KI-0399EPGH},2011-05-31 10:42:06,AES1373,PC-6944,R:\;R:\AES1373,Connect,15
4,{N1J2-U0KJ64RK-2745DCQD},2011-05-31 10:09:02,AES1373,PC-6944,R:\;R:\AES1373,Connect,13
...,...,...,...,...,...,...,...
47544,{J5Z0-O8JO04MS-1915REAH},2011-01-11 04:20:58,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect,4
47545,{J0G7-L5WK98XP-7403ABEV},2011-01-11 03:44:17,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect,25
47546,{O9M2-A0LZ88NQ-8497GZNB},2011-01-11 03:35:11,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect,1
47547,{T1X3-R8VM32JD-4082IPBP},2011-01-11 03:06:22,ZKP0542,PC-2083,R:\;R:\ZKP0542;R:\57Vl3S2;R:\52N7xr4;R:\38v2d7...,Connect,4


#**Extract file features**

#### TODO: Find more elegant way to extract features (pivot_table, another way to groupby, etc. )

Statistical features are extracted on all day operations, not on specific file type/removable media etc.

In [ ]:
def process_file(file_df):
    file_df['file_extension'] = file_df['filename'].apply(lambda x: x.split('.')[-1])
    file_df['file_length'] = file_df['content'].apply(lambda x: len(x.split()))
    file_df['file_depth'] = file_df['filename'].apply(lambda x: len(x.split('\\')))
    return file_df

In [ ]:
file_df = pd.read_csv(os.path.join(DataSetPath, "file.csv"))
file_df['date'] = pd.to_datetime(file_df['date'])
file_df.head()

,id,date,user,pc,filename,activity,to_removable_media,from_removable_media,content
0,{G5X3-Z3TT87MZ-1125ZWHW},2010-01-02 19:14:34,VRP0267,PC-8461,R:\ICA5T4Q8.doc,File Open,False,True,D0-CF-11-E0-A1-B1-1A-E1 devised mine officiall...
1,{U5N0-D9KT82SH-1353KTHL},2010-01-02 19:15:01,VRP0267,PC-8461,C:\GPH0932\NFR12BZA.doc,File Open,False,False,D0-CF-11-E0-A1-B1-1A-E1 islands consistently o...
2,{J8K3-E7NP02GP-8515JPPH},2010-01-02 23:06:10,VRP0267,PC-1985,R:\45L8257\XOCNOM4O.pdf,File Write,True,False,25-50-44-46-2D greater he did respectable peop...
3,{W5T9-V3BA42RO-8223ANRJ},2010-01-03 09:05:18,MAB1340,PC-1272,R:\29Z1tL4\F8DHK5K7.doc,File Open,False,True,D0-CF-11-E0-A1-B1-1A-E1 whether alternative in...
4,{Q0K4-O9CI90HJ-9787SZTW},2010-01-03 09:16:32,MAB1340,PC-1272,R:\29Z1tL4\F8DHK5K7.doc,File Write,True,False,D0-CF-11-E0-A1-B1-1A-E1 siding purpose met not...


In [ ]:
tmp_file_df = process_file(file_df)
tmp_file_df['weekday'] = tmp_file_df.date.dt.weekday
tmp_file_df['day'] = pd.to_datetime(tmp_file_df['date']).dt.normalize()
tmp_file_df['hour'] = tmp_file_df['date'].dt.hour

tmp_file_df['is_weekend'] = tmp_file_df['weekday'].apply(is_weekend)
tmp_file_df['is_working_hour'] = tmp_file_df['hour'].apply(is_working_hour)
tmp_file_df['is_personal_pc'] = tmp_file_df.apply(is_personal_pc, args=[user_pcs_df], axis=1)
tmp_file_df.head()

,id,date,user,pc,filename,activity,to_removable_media,from_removable_media,content,file_extension,file_length,file_depth,weekday,day,hour,is_weekend,is_working_hour,is_personal_pc
0,{G5X3-Z3TT87MZ-1125ZWHW},2010-01-02 19:14:34,VRP0267,PC-8461,R:\ICA5T4Q8.doc,File Open,False,True,D0-CF-11-E0-A1-B1-1A-E1 devised mine officiall...,doc,52,2,5,2010-01-02,19,weekend,after_hours,other_pc
1,{U5N0-D9KT82SH-1353KTHL},2010-01-02 19:15:01,VRP0267,PC-8461,C:\GPH0932\NFR12BZA.doc,File Open,False,False,D0-CF-11-E0-A1-B1-1A-E1 islands consistently o...,doc,41,3,5,2010-01-02,19,weekend,after_hours,other_pc
2,{J8K3-E7NP02GP-8515JPPH},2010-01-02 23:06:10,VRP0267,PC-1985,R:\45L8257\XOCNOM4O.pdf,File Write,True,False,25-50-44-46-2D greater he did respectable peop...,pdf,51,3,5,2010-01-02,23,weekend,after_hours,other_pc
3,{W5T9-V3BA42RO-8223ANRJ},2010-01-03 09:05:18,MAB1340,PC-1272,R:\29Z1tL4\F8DHK5K7.doc,File Open,False,True,D0-CF-11-E0-A1-B1-1A-E1 whether alternative in...,doc,51,3,6,2010-01-03,9,weekend,working_hour,personal_pc
4,{Q0K4-O9CI90HJ-9787SZTW},2010-01-03 09:16:32,MAB1340,PC-1272,R:\29Z1tL4\F8DHK5K7.doc,File Write,True,False,D0-CF-11-E0-A1-B1-1A-E1 siding purpose met not...,doc,47,3,6,2010-01-03,9,weekend,working_hour,personal_pc


In [ ]:
groupby_cols = ['day', 'user', 'activity', 'file_extension',
                'from_removable_media', 'is_working_hour', 'is_personal_pc']
user_daily_counts = pd.DataFrame({"count" : tmp_file_df.groupby(groupby_cols).size()}).reset_index()
user_daily_counts = user_daily_counts.set_index(groupby_cols).unstack(['activity', 'file_extension',
                                     'from_removable_media', 'is_working_hour', 'is_personal_pc'])
user_daily_counts.columns = ['{}_{}_{}_{}_{}_{}'.format(a,b,c,d,e,f) for a,b,c,d,e,f in user_daily_counts.columns]

user_daily_counts.fillna(0, inplace=True)
user_daily_counts = user_daily_counts.reset_index()
user_daily_counts

,day,user,count_File Open_doc_False_after_hours_other_pc,count_File Open_doc_True_after_hours_other_pc,count_File Write_pdf_False_after_hours_other_pc,count_File Open_doc_True_working_hour_personal_pc,count_File Write_doc_False_working_hour_personal_pc,count_File Open_txt_True_working_hour_personal_pc,count_File Open_pdf_True_working_hour_other_pc,count_File Write_doc_False_working_hour_other_pc,...,count_File Open_pdf_False_after_hours_personal_pc,count_File Write_txt_False_after_hours_personal_pc,count_File Open_exe_True_after_hours_supervisor_pc,count_File Open_txt_False_after_hours_personal_pc,count_File Open_zip_True_after_hours_personal_pc,count_File Write_zip_False_after_hours_personal_pc,count_File Open_jpg_False_after_hours_personal_pc,count_File Write_exe_False_after_hours_personal_pc,count_File Open_exe_False_working_hour_other_pc,count_File Open_exe_True_working_hour_supervisor_pc
0,2010-01-02,VRP0267,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-03,MAB1340,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-01-03,VCF1602,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-01-03,VRP0267,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-04,AES1373,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9047,2011-06-01,MAB1340,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9048,2011-06-01,PAA0121,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9049,2011-06-01,RCG0584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9050,2011-06-01,RHG0882,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_daily_file_stats = tmp_file_df.groupby(["day", "user"])['file_length', 'file_depth'].mean().reset_index()
user_daily_file = user_daily_counts.merge(user_daily_file_stats, on=['day', 'user'])

user_daily_file

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,day,user,count_File Open_doc_False_after_hours_other_pc,count_File Open_doc_True_after_hours_other_pc,count_File Write_pdf_False_after_hours_other_pc,count_File Open_doc_True_working_hour_personal_pc,count_File Write_doc_False_working_hour_personal_pc,count_File Open_txt_True_working_hour_personal_pc,count_File Open_pdf_True_working_hour_other_pc,count_File Write_doc_False_working_hour_other_pc,...,count_File Open_exe_True_after_hours_supervisor_pc,count_File Open_txt_False_after_hours_personal_pc,count_File Open_zip_True_after_hours_personal_pc,count_File Write_zip_False_after_hours_personal_pc,count_File Open_jpg_False_after_hours_personal_pc,count_File Write_exe_False_after_hours_personal_pc,count_File Open_exe_False_working_hour_other_pc,count_File Open_exe_True_working_hour_supervisor_pc,file_length,file_depth
0,2010-01-02,VRP0267,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.000000,2.666667
1,2010-01-03,MAB1340,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.250000,3.000000
2,2010-01-03,VCF1602,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.333333,3.000000
3,2010-01-03,VRP0267,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.500000,2.833333
4,2010-01-04,AES1373,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.100000,2.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9047,2011-06-01,MAB1340,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.250000,3.000000
9048,2011-06-01,PAA0121,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.500000,2.500000
9049,2011-06-01,RCG0584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,3.000000
9050,2011-06-01,RHG0882,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.666667,3.000000


In [ ]:
user_daily_file.to_csv(os.path.join(RootPath,'user_daily_file.csv'),index = False)

#**HTTP Features**

Hacktivist category is not in columns find out why

In [ ]:
def process_http(http_df):
    http_df['url_length'] = http_df['url'].apply(lambda x: len(x))
    http_df['url_depth'] = http_df['url'].apply(lambda x: len(x.split('/')) - 1)
    http_df['http_content_length'] = http_df['content'].apply(lambda x: len(x.split()))
    return http_df

def parse_http(x):
    if x.sld in ['dropbox.com', 'drive.google.com', 'mega.co.nz', 'account.live.com']:
        return "cloud_webvisit"
    
    elif x.sld in ['wikileaks.org','freedom.press','theintercept.com']:
        return "hacktivist_webvisit"
    
    elif x.sld in ['facebook.com','twitter.com','plus.google.com','instagr.am','instagram.com',
                        'flickr.com','linkedin.com','reddit.com','about.com','youtube.com','pinterest.com',
                        'tumblr.com','quora.com','vine.co','match.com','t.co']:
        return "social_webvisit"
    
    elif x.sld in ['indeed.com','monster.com', 'careerbuilder.com','simplyhired.com']:
        return "jobsearch_webvisit"
   
    elif ('job' in x.url and ('hunt' in x.url or 'search' in x.url)):
          return "jobsearch_webvisit"
    else:
        return "webvisit"

In [ ]:
http_df = pd.read_csv(os.path.join(DataSetPath,"http.csv"))
http_df['date'] = pd.to_datetime(http_df['date'])
http_df['day'] = pd.to_datetime(http_df['date']).dt.normalize()
http_df['activity'] = http_df.apply(parse_http, axis=1)
http_df = process_http(http_df)
http_df.tail(10)

,id,date,user,pc,url,content,sld,day,activity,url_length,url_depth,http_content_length
5673143,{J2Y8-C1IQ27VT-2564JMKZ},2011-06-01 19:56:34,JGW0820,PC-4239,http://twimg.com/Moe_Berg/tashker/pnegbbaznahs...,history finally food completely trunk dominati...,twimg.com,2011-06-01,webvisit,80,5,75
5673144,{C8F5-H1RX52SK-0666QJPW},2011-06-01 19:58:42,HCV0286,PC-6054,http://filestube.com/Eduard_Streltsov/lebedeva...,someone subsequently police bounced truck as t...,filestube.com,2011-06-01,webvisit,120,5,42
5673145,{H7Z7-L7UO85TO-4601RYYM},2011-06-01 19:59:00,HCV0286,PC-6054,http://filestube.com/Eduard_Streltsov/lebedeva...,without remake arrest author recurring season ...,filestube.com,2011-06-01,webvisit,120,5,59
5673146,{E7A8-F3QC40UI-4644DRDE},2011-06-01 20:03:45,NCB0657,PC-7928,http://wikipedia.org/Maya_MIA_album/rusko/obju...,production 1990s new more koker held everythin...,wikipedia.org,2011-06-01,webvisit,103,5,54
5673147,{G7I7-L1RL06WV-4772EZTK},2011-06-01 20:04:16,NCB0657,PC-7928,http://authorize.net/LaplaceRungeLenz_vector/l...,stated fibres drawn seats agreeing 24 sebastia...,authorize.net,2011-06-01,webvisit,73,5,59
5673148,{D5A8-I0BA65BR-7702TYJU},2011-06-01 20:06:21,NCB0657,PC-7928,http://authorize.net/LaplaceRungeLenz_vector/l...,50000 three tony anfield keep laing circumfere...,authorize.net,2011-06-01,webvisit,73,5,38
5673149,{R4R3-D6WN96CP-9717MMBB},2011-06-01 20:08:10,NCB0657,PC-7928,http://egotastic.com/Final_Fantasy_XI/vanadiel...,on regional teams problem granted without enti...,egotastic.com,2011-06-01,webvisit,80,5,52
5673150,{W3Y2-H4UP06DQ-5743ADQV},2011-06-01 20:09:07,NCB0657,PC-7928,http://authorize.net/LaplaceRungeLenz_vector/l...,one 5000 trials ice reinforced tier because mi...,authorize.net,2011-06-01,webvisit,73,5,37
5673151,{X5U1-E6QJ09BJ-8331SHPX},2011-06-01 20:09:25,FGA0659,PC-5441,http://domaintools.com/Indian_Head_eagle/gaude...,own bloeme haven opened disease tried unknown ...,domaintools.com,2011-06-01,webvisit,88,5,59
5673152,{F1N3-J7XQ24SW-8167JIEB},2011-06-01 20:22:48,FGA0659,PC-5441,http://monster.com/WboUhagvat1180707852.html,sales engineer management develop on-line deve...,monster.com,2011-06-01,jobsearch_webvisit,44,3,40


In [ ]:
http_df.activity.value_counts()

webvisit               5380196
social_webvisit         204097
jobsearch_webvisit       69712
cloud_webvisit           19083
hacktivist_webvisit         65
Name: activity, dtype: int64

In [ ]:
groupby_cols = ['activity', 'day', 'user']
x = pd.DataFrame({"count" : http_df.groupby(groupby_cols).size()}).reset_index()

x.activity.value_counts()

webvisit               62350
social_webvisit        33488
jobsearch_webvisit     13616
cloud_webvisit          7629
hacktivist_webvisit       63
Name: activity, dtype: int64

In [ ]:
groupby_cols = ['day', 'user', 'activity']

http_daily_counts = pd.DataFrame({"count" : http_df.groupby(groupby_cols).size()}).reset_index()
http_daily_counts = http_daily_counts.set_index(groupby_cols).unstack(['activity'])

http_daily_counts.columns = ['{}_{}'.format(a,b) for a,b in http_daily_counts.columns]

http_daily_counts.fillna(0, inplace=True)
http_daily_counts = http_daily_counts.reset_index()
http_daily_counts

,day,user,count_cloud_webvisit,count_hacktivist_webvisit,count_jobsearch_webvisit,count_social_webvisit,count_webvisit
0,2010-01-02,ELT1370,0.0,0.0,0.0,1.0,9.0
1,2010-01-02,ESM1828,0.0,0.0,0.0,0.0,153.0
2,2010-01-02,GVM1337,2.0,0.0,0.0,2.0,91.0
3,2010-01-02,ILM1441,0.0,0.0,0.0,0.0,27.0
4,2010-01-02,IRH1224,0.0,0.0,0.0,9.0,86.0
...,...,...,...,...,...,...,...
62351,2011-06-01,UCC0395,0.0,0.0,18.0,3.0,122.0
62352,2011-06-01,WBK1077,0.0,0.0,5.0,1.0,156.0
62353,2011-06-01,WCW1013,0.0,0.0,0.0,1.0,94.0
62354,2011-06-01,WOS1834,0.0,0.0,0.0,7.0,155.0


In [ ]:
http_daily_stats = http_df.groupby(["day", "user"])['url_length', 'url_depth', 'http_content_length']\
                                    .mean().reset_index()
http_daily_file = http_daily_counts.merge(http_daily_stats, on=['day', 'user'])

http_daily_file

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,day,user,count_cloud_webvisit,count_hacktivist_webvisit,count_jobsearch_webvisit,count_social_webvisit,count_webvisit,url_length,url_depth,http_content_length
0,2010-01-02,ELT1370,0.0,0.0,0.0,1.0,9.0,87.500000,5.000000,53.000000
1,2010-01-02,ESM1828,0.0,0.0,0.0,0.0,153.0,98.660131,5.000000,49.117647
2,2010-01-02,GVM1337,2.0,0.0,0.0,2.0,91.0,95.705263,5.000000,50.042105
3,2010-01-02,ILM1441,0.0,0.0,0.0,0.0,27.0,97.518519,5.000000,47.740741
4,2010-01-02,IRH1224,0.0,0.0,0.0,9.0,86.0,84.547368,5.000000,48.389474
...,...,...,...,...,...,...,...,...,...,...
62351,2011-06-01,UCC0395,0.0,0.0,18.0,3.0,122.0,93.391608,5.000000,50.279720
62352,2011-06-01,WBK1077,0.0,0.0,5.0,1.0,156.0,87.987654,4.839506,49.938272
62353,2011-06-01,WCW1013,0.0,0.0,0.0,1.0,94.0,89.526316,5.000000,50.021053
62354,2011-06-01,WOS1834,0.0,0.0,0.0,7.0,155.0,96.746914,5.000000,47.308642


In [ ]:
http_daily_file.to_csv(os.path.join(RootPath,'user_daily_http.csv'),index = False)

# LOGON

In [ ]:
# statistical features
def add_session_time(df):

  grouped_df = df.groupby('pc')
  df_list = []

  for key,_ in grouped_df:
    group = grouped_df.get_group(key)
    group = group.sort_values(['date'] , ascending = [False])

    difference = group["date"].diff().apply(lambda x: x/np.timedelta64(1, 'm')*-1).fillna(0).astype('int64')
    group['session_time'] = pd.Series(difference)
    group = group[group['activity']=='Logon']

    df_list.append(group)

  df_time = pd.concat(df_list, ignore_index = True, axis = 0)

  return df_time

def process_logon(df):
  df = df.copy()
  df_time = add_session_time(df)

  return df_time

In [ ]:
logon_df = pd.read_csv(os.path.join(DataSetPath,'logon.csv'))
logon_df['date'] = pd.to_datetime(logon_df['date'])
logon_df.head(5)

,id,date,user,pc,activity
0,{U0U5-J2AX64NN-7430KKNP},2010-01-02 07:27:00,VRP0267,PC-5687,Logon
1,{B0G5-P2XR80RE-6009PRBI},2010-01-02 07:34:00,IRH1224,PC-4126,Logon
2,{R3Z9-U5YR87BS-2979BOUZ},2010-01-02 07:40:00,ELT1370,PC-1929,Logon
3,{X0C4-S3RJ20AY-4832PPRI},2010-01-02 07:41:00,JMM0613,PC-7127,Logon
4,{U3L5-D8ZD99IK-6715VWBI},2010-01-02 07:45:00,GVM1337,PC-6338,Logon


In [ ]:
tmp_logon_df = process_logon(logon_df)
tmp_logon_df['weekday'] = tmp_logon_df.date.dt.weekday
tmp_logon_df['hour'] = tmp_logon_df['date'].dt.hour
tmp_logon_df['day'] = pd.to_datetime(tmp_logon_df['date']).dt.normalize()

In [ ]:
tmp_logon_df['is_weekend'] = tmp_logon_df['weekday'].apply(is_weekend)
tmp_logon_df['is_working_hour'] = tmp_logon_df['hour'].apply(is_working_hour)
tmp_logon_df['is_personal_pc'] = tmp_logon_df.apply(is_personal_pc, args=[user_pcs_df], axis=1)

tmp_logon_df.head(10)

,id,date,user,pc,activity,session_time,weekday,hour,day,is_weekend,is_working_hour,is_personal_pc
0,{T9D7-E2PX01FV-2444VOLU},2010-10-08 21:57:59,CWW1120,PC-0010,Logon,5,4,21,2010-10-08,weekday,after_hours,other_pc
1,{Q7A9-F8SB19IG-9037JXKW},2010-08-20 00:20:15,GKW0043,PC-0010,Logon,11,4,0,2010-08-20,weekday,after_hours,other_pc
2,{W6S6-U0AI84IL-4615JFOG},2010-07-14 04:01:40,LAH0463,PC-0010,Logon,18,2,4,2010-07-14,weekday,after_hours,other_pc
3,{V8B7-I8PZ45AA-0851HKSG},2010-04-27 18:11:53,ZEH0685,PC-0010,Logon,2,1,18,2010-04-27,weekday,working_hour,other_pc
4,{E6R2-Y4GK75JZ-4790TZCL},2010-04-06 04:17:59,ZEH0685,PC-0010,Logon,17,1,4,2010-04-06,weekday,after_hours,other_pc
5,{M7B0-G4CL94SP-3950YMTX},2010-03-16 03:48:48,SIS0042,PC-0010,Logon,19,1,3,2010-03-16,weekday,after_hours,other_pc
6,{L3B6-V9TJ75EX-3965WUZD},2010-03-08 22:42:32,ZEH0685,PC-0010,Logon,8,0,22,2010-03-08,weekday,after_hours,other_pc
7,{B6K5-R4TK80OD-5503TMGZ},2010-02-19 17:59:36,DCC1119,PC-0010,Logon,15,4,17,2010-02-19,weekday,working_hour,other_pc
8,{U4R9-Q5RI27QE-4135BMWF},2010-02-15 23:38:37,VRP0267,PC-0010,Logon,5,0,23,2010-02-15,weekday,after_hours,other_pc
9,{X5S2-D7GS52GH-3365PWFO},2011-02-28 20:19:58,CHW0919,PC-0014,Logon,5,0,20,2011-02-28,weekday,after_hours,other_pc


In [ ]:
tmp_logon_df.is_personal_pc.value_counts(dropna=False)

personal_pc      81026
other_pc         15758
supervisor_pc       29
Name: is_personal_pc, dtype: int64

In [ ]:
user_pcs_df[user_pcs_df["pc"]=='PC-0010']

,pc,user
393,PC-0010,BWW0751


In [ ]:
groupby_cols = ['day', 'user', 'activity', 'is_working_hour', 'is_personal_pc' ]
user_daily_logon_counts = pd.DataFrame({"count" : tmp_logon_df.groupby(groupby_cols).size()}).reset_index()
user_daily_logon_counts = user_daily_logon_counts.set_index(groupby_cols).unstack(['activity', 'is_working_hour', 'is_personal_pc'])
user_daily_logon_counts.columns = ['{}_{}_{}_{}'.format(a,b,c,d) for a,b,c,d in user_daily_logon_counts.columns]

user_daily_logon_counts.fillna(0, inplace=True)
user_daily_logon_counts = user_daily_logon_counts.reset_index()
user_daily_logon_counts

,day,user,count_Logon_working_hour_other_pc,count_Logon_working_hour_personal_pc,count_Logon_after_hours_other_pc,count_Logon_after_hours_personal_pc,count_Logon_after_hours_supervisor_pc,count_Logon_working_hour_supervisor_pc
0,2010-01-02,ELT1370,1.0,2.0,0.0,0.0,0.0,0.0
1,2010-01-02,ESM1828,1.0,1.0,0.0,0.0,0.0,0.0
2,2010-01-02,GVM1337,0.0,2.0,0.0,0.0,0.0,0.0
3,2010-01-02,ILM1441,1.0,1.0,0.0,0.0,0.0,0.0
4,2010-01-02,IRH1224,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
62421,2011-06-01,WBK1077,0.0,2.0,0.0,0.0,0.0,0.0
62422,2011-06-01,WCW1013,0.0,1.0,0.0,0.0,0.0,0.0
62423,2011-06-01,WOS1834,0.0,1.0,0.0,0.0,0.0,0.0
62424,2011-06-01,YBD1929,0.0,1.0,0.0,0.0,0.0,0.0


**Statistical Features**

In [ ]:
user_daily_logon_stats = tmp_logon_df.groupby(["day", "user"])[ 'session_time'].mean().reset_index()
user_daily_logon_stats.head(10)

,day,user,session_time
0,2010-01-02,ELT1370,163.666667
1,2010-01-02,ESM1828,276.500000
2,2010-01-02,GVM1337,277.000000
3,2010-01-02,ILM1441,261.500000
4,2010-01-02,IRH1224,686.000000
5,2010-01-02,JMM0613,490.000000
6,2010-01-02,KRC0098,277.500000
7,2010-01-02,VRP0267,138.000000
8,2010-01-03,DLW1705,619.000000
9,2010-01-03,GVM1337,267.000000


**Merge**

In [ ]:
user_daily_logon = user_daily_logon_counts.merge(user_daily_logon_stats, on=['day', 'user'])
user_daily_logon.head(10)

,day,user,count_Logon_working_hour_other_pc,count_Logon_working_hour_personal_pc,count_Logon_after_hours_other_pc,count_Logon_after_hours_personal_pc,count_Logon_after_hours_supervisor_pc,count_Logon_working_hour_supervisor_pc,session_time
0,2010-01-02,ELT1370,1.0,2.0,0.0,0.0,0.0,0.0,163.666667
1,2010-01-02,ESM1828,1.0,1.0,0.0,0.0,0.0,0.0,276.500000
2,2010-01-02,GVM1337,0.0,2.0,0.0,0.0,0.0,0.0,277.000000
3,2010-01-02,ILM1441,1.0,1.0,0.0,0.0,0.0,0.0,261.500000
4,2010-01-02,IRH1224,0.0,1.0,0.0,0.0,0.0,0.0,686.000000
5,2010-01-02,JMM0613,0.0,1.0,0.0,0.0,0.0,0.0,490.000000
6,2010-01-02,KRC0098,0.0,2.0,0.0,0.0,0.0,0.0,277.500000
7,2010-01-02,VRP0267,0.0,1.0,4.0,0.0,0.0,0.0,138.000000
8,2010-01-03,DLW1705,0.0,1.0,0.0,0.0,0.0,0.0,619.000000
9,2010-01-03,GVM1337,0.0,2.0,0.0,0.0,0.0,0.0,267.000000


In [ ]:
user_daily_logon.to_csv( os.path.join(RootPath,'user_daily_logon.csv') ,index = False)

###**Labeling Dataset**

In [ ]:
def is_malicious(x , mal_users_events):
    x = list(x) 
    user = x[1]
    min_date = x[0]
    max_date = x[0]
    if(user not in mal_users_events):
        return 0
    else:
        user_df = mal_users_events[user]
        mal_events_count = user_df[(user_df['date'] >= min_date) & 
                                   (user_df['date'] <= max_date)].shape[0]
        return int(mal_events_count > 0)

def get_user_from_path(path):
    user = path.split('/')[-1].split('.csv')[0].split('-')[-1]
    return user

def read_answer_file(path):
    df = pd.read_csv(path, header=None, usecols=[0,1,2,3,4]).rename(columns={0: 'action',
                                                                            1: 'ID',
                                                                            2: 'date',
                                                                            3: 'user',
                                                                            4: 'pc'})
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.date
    return df

def read_malicious_events(answers_dir, dataset_ver=5.2):
    mal_event_paths = glob.glob(os.path.join(answers_dir, f'r{dataset_ver}-*/*'))
    mal_users = [get_user_from_path(p) for p in mal_event_paths]
    mal_user_events = {user: read_answer_file(path)
                       for (user, path) in zip(mal_users, mal_event_paths)}
    return mal_user_events


In [ ]:
mal_users_events = read_malicious_events(AnswersPATH)

In [ ]:
mal_users_events = read_malicious_events(AnswersPATH)